# Solution Overview

In [1]:
import pandas as pd

RETROFIT_PATH = '/Users/pengfeicheng/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/FLECCS/NPV-update/'
series_order = ['ERCOT', 'CAISO', 'NYISO', 'MISO-W', 'PJM-W', 'BaseCaseTax', 'HighWindTax', 'HighSolarTax', 'WinterNYTax']

## Read meta data

In [2]:
df = pd.read_csv(RETROFIT_PATH + 'START-5-NG-383_meta.csv')

df = df[['CO2_credit', 'scenario_name', 'solve_time', 'sorbent_amount', 'gap']]

df['scenario_name'] = df['scenario_name'].apply(lambda x: x if '_' not in x else x.split('_')[-1])
df.head()

,CO2_credit,scenario_name,solve_time,sorbent_amount,gap
0,150.0,NYISO,1597.0,2271.306531,8.372394e-03
1,225.0,NYISO,634.0,2988.568680,0.000000e+00
2,300.0,NYISO,493.0,2988.568680,0.000000e+00
3,150.0,PJM-W,1708.0,2271.306531,7.458370e-03
4,225.0,PJM-W,625.0,2988.568680,1.001451e-15


In [5]:
df_sol_time = pd.DataFrame()
df_dac = pd.DataFrame()
df_gap = pd.DataFrame()

for _,row in df.iterrows():
    df_sol_time.loc[row['CO2_credit'], row['scenario_name']] = row['solve_time']
    df_dac.loc[row['CO2_credit'], row['scenario_name']] = row['sorbent_amount']
    df_gap.loc[row['CO2_credit'], row['scenario_name']] = row['gap']

dfs = [df_sol_time, df_dac, df_gap]
for _df in dfs:
    _df.sort_index(inplace=True)

df_dac
df_sol_time
df_gap
# df_dac.to_excel('dac.xlsx')
# df_sol_time.to_excel('sol_time.xlsx')
# df_gap.to_excel('gap.xlsx')

In [23]:
KEY = 'NG-383'
ADDRESS = "/Users/pengfeicheng/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/FLECCS/NPV-update/"
NG_383_folders = [name for name in os.listdir(ADDRESS) if KEY in name and os.path.isdir(ADDRESS + name) and 'START-5-' not in name]

In [24]:
KEY = 'START-5-NG-383'
ADDRESS = "/Users/pengfeicheng/Library/CloudStorage/OneDrive-GeorgiaInstituteofTechnology/FLECCS/NPV-update/"
START_5_NG_383_folders = [name for name in os.listdir(ADDRESS) if KEY in name and os.path.isdir(ADDRESS + name)]

In [20]:
def gen_retrofit_df_f(func, folders):

    df = pd.DataFrame()

    for folder in folders:

        CO2_price = folder.split('month-')[1].split('-')[0]
        scenario = '-'.join(folder.split('month-')[1].split('-')[1:]).split('_')[-1]
        df.loc[CO2_price, scenario] = func(folder)

    df = df[series_order].sort_index()

    return df.round(2)

## get start up times

In [ ]:
def read_z(folder_name):
    folder_path = RETROFIT_PATH + folder_name

    table_name = 'results_binary_vars.csv'

    df_binary = pd.read_csv(folder_path + '/' + table_name)
    z = sum(df_binary['z'])

    return int(z)

df = gen_retrofit_df_f(read_z, NG_383_folders)
df

,ERCOT,CAISO,NYISO,MISO-W,PJM-W,BaseCaseTax,HighWindTax,HighSolarTax,WinterNYTax
100,96.0,137.0,101.0,52.0,102.0,0.0,251.0,292.0,0.0
150,89.0,109.0,93.0,52.0,84.0,0.0,0.0,0.0,0.0
225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Operation hours

In [ ]:
def read_y_sum(folder_name):
    folder_path = RETROFIT_PATH + folder_name

    table_name = 'results_binary_vars.csv'

    df_binary = pd.read_csv(folder_path + '/' + table_name)
    y = sum(df_binary['y'])

    return int(y)

In [ ]:
df_y_1 = gen_retrofit_df_f(read_y_sum, NG_383_folders)
df_y_1

,ERCOT,CAISO,NYISO,MISO-W,PJM-W,BaseCaseTax,HighWindTax,HighSolarTax,WinterNYTax
100,3914.0,6304.0,4607.0,4638.0,6280.0,8640.0,6779.0,6653.0,8640.0
150,4060.0,4544.0,3601.0,3084.0,5096.0,8640.0,8640.0,8640.0,8640.0
225,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0
300,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0


In [ ]:
df_y_2 = gen_retrofit_df_f(read_y_sum, START_5_NG_383_folders)
df_y_2

,ERCOT,CAISO,NYISO,MISO-W,PJM-W,BaseCaseTax,HighWindTax,HighSolarTax,WinterNYTax
100,4752.0,6303.0,3915.0,3751.0,6727.0,8640.0,8528.0,8640.0,8640.0
150,6956.0,5300.0,7700.0,5279.0,8640.0,8640.0,8640.0,8640.0,8640.0
225,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0
300,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0,8640.0


In [ ]:
df_y_2 - df_y_1

,ERCOT,CAISO,NYISO,MISO-W,PJM-W,BaseCaseTax,HighWindTax,HighSolarTax,WinterNYTax
100,838.0,-1.0,-692.0,-887.0,447.0,0.0,1749.0,1987.0,0.0
150,2896.0,756.0,4099.0,2195.0,3544.0,0.0,0.0,0.0,0.0
225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## get DAC size

In [ ]:
def read_dac_size(folder_name):
    folder_path = RETROFIT_PATH + folder_name

    table_name = 'results_DAC_costing.csv'

    df_dac = pd.read_csv(folder_path + '/' + table_name)
    return df_dac.loc[0, 'x_sorbent_m']

In [ ]:
df = gen_retrofit_df_f(read_dac_size, NG_383_folders)
df

,ERCOT,CAISO,NYISO,MISO-W,PJM-W,BaseCaseTax,HighWindTax,HighSolarTax,WinterNYTax
100,0.0,0.0,0.0,0.0,730.1,2271.3,730.1,0.0,2271.3
150,730.1,730.1,0.0,0.0,730.1,2988.6,2988.6,2988.6,2988.6
225,2988.6,2988.6,2988.6,2988.6,2988.6,2988.6,2988.6,2988.6,2988.6
300,2988.6,2988.6,2988.6,2988.6,2988.6,2988.6,2988.6,2988.6,2988.6


## get avg. load

In [21]:
def read_avg_load(folder_name):
    folder_path = RETROFIT_PATH + folder_name

    table_name = 'results_power.csv'

    df_power = pd.read_csv(folder_path + '/' + table_name)
    return df_power['x_load_factor'].mean()

In [ ]:
df_avg_load_1 = gen_retrofit_df_f(read_avg_load, NG_383_folders)
df_avg_load_1

,ERCOT,CAISO,NYISO,MISO-W,PJM-W,BaseCaseTax,HighWindTax,HighSolarTax,WinterNYTax
100,38.0,62.3,45.6,49.7,64.3,91.7,59.1,54.3,93.3
150,39.0,43.8,34.7,31.8,52.4,92.2,88.4,88.8,94.1
225,69.9,70.6,66.5,65.9,77.3,93.5,89.7,90.1,94.7
300,69.9,70.6,66.6,68.3,84.6,95.4,90.6,93.0,98.0


In [28]:
df_avg_load_2 = gen_retrofit_df_f(read_avg_load, START_5_NG_383_folders)
df_avg_load_2

,ERCOT,CAISO,NYISO,MISO-W,PJM-W,BaseCaseTax,HighWindTax,HighSolarTax,WinterNYTax
100,44.81,65.52,39.19,40.58,68.61,91.67,87.06,87.45,93.28
150,58.44,51.10,60.98,46.11,75.51,92.25,88.44,88.77,94.09
225,69.91,70.60,66.53,65.86,77.31,93.51,89.74,90.06,94.69
300,69.92,70.60,66.62,68.32,84.57,95.37,90.62,92.95,98.03


In [ ]:
df_avg_load_2 - df_avg_load_1

,ERCOT,CAISO,NYISO,MISO-W,PJM-W,BaseCaseTax,HighWindTax,HighSolarTax,WinterNYTax
100,6.8,3.2,-6.4,-9.1,4.3,0.0,28.0,33.2,0.0
150,19.4,7.3,26.3,14.3,23.1,0.0,0.0,0.0,0.0
225,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
300,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
